In [1]:
# ! git clone https://github.com/roshan-research/hazm.git

In [ ]:
# ! python hazm/setup.py install

In [ ]:
import numpy as np
import nltk
import re
import string
import warnings
import gensim
from sklearn.metrics.pairwise import cosine_similarity
from configparser import ConfigParser
from functools import reduce
from gensim.models import Doc2Vec
from hazm.Embedding import SentEmbedding
from hazm import *

In [ ]:
text = 'پدرم یک کشاورز بودمثل همه که مجبور به این کار بودنالبته شغل‌شو اونطوری شروع نکردسیستم میگه در معرض فشار قرار گرفتینه، مشکلی ندارهاز مخمصه عبور می‌کنیمباید خاموشش کنم، کوپرهمشو خاموش میکنم!نباید چیزی رو خاموش کنیمبابا؟شرمنده، مورف. برگرد بخوابفکر کردم تو روحیگلم روح و اینا وجود ندارهبابابزرگ میگه تو میتونی ارواح رو بگیری...نه، این بخاطر اینکه بابابزرگدیر یا زود خودش از اونا میشه. برو بخوابداشتی خواب سقوط رو میدیدی؟برگرد برو بخواب، مورفمحصولات گندم از بین رفته بودآفت اومده بود و ما مجبور به سوزوندنش بودیمهنوز زمین ذرت داشتیمچندین قطعه مزرعه ذرت داشتیمولی بیشتر وقتا گرد و غبار می‌اومدگمونم نمی‌تونم توضیح بدمطوفان گرد و خاک دائماً می‌اومدهمین، یه عالمه غبار و کثافت...ما خودمونو با پارچه‌هایی می‌پوشوندیمبعضی وقتا روی بینی و دهانمون می‌ذاشتیمکه نتونیم گرد و خاک زیادی رو بدمیمخب زمانی که بشقاب‌ها رو می‌چیدیماونا رو وارونه روی میز می‌ذاشتیملیوان‌ها، بشقاب‌ها، همه چی رو وارونه می‌کردیمبجنبین. مورف، عجله کن...ساعت توی مزرعه، واسه سم ‌پاشیبرای منطقه به روی چشمروی میز نه، مورفبابا، میتونی اینو درست کنی؟چه بلایی سر فضاپیمام آوردی؟من نبودمبذار حدس بزنم. جناب روح این کارو کردبه قفسه کتابم ضربه میزدهمچنین کتاب‌ها رو می‌ندازه زمینکله‌خر، روح وجود خارجی ندارهدربارش تحقیق کردم، بهش میگن شَـبـحبابا، بهش بگوخب، زیادم علمی نیست، مورفخودت گفتی علم درباره پذیرفتنِچیزاییه که ما از اونا نمی‌دونیم!حالا جوابشو بدهمراقب وسایل‌هامون باشکوپخیلی خب، مورف، میخوای علمی حرف بزنی؟،حالا نگو که از روح و اینا می‌ترسینه، تو باید فراتر از این باشی...باید وقایع رو ضبط کنی، آنالیزشون کنیبعد بطور منطقی "چرا و چگونه" این روپیدا کنی و نتایج رو بهم نشون بدی. قبول؟قبولخیلی خبروز خوبی تو مدرسه داشته باشیوایسا،مدرسه جلسه‌ی اولیا تشکیل دادهاولیا، نه اولیای اولیاآروم پسرماون طوفان گرد و خبار نیستنسلون داره محصولاتشو می‌سوزونهآفت؟گفتن که این آخرین برداشتِمحصول واسه بامیه هست. تموم شدبهتر بود مثل ما ذرت پرورش میدادبا خانم هنلی خوب رفتار کناون مجردهاین یعنی چه؟تولید مثل، برای ادامه حیاتشروع کن مثل بقیه سخت کار کنی، جوونچرا سرتو نمی‌ندازی پایین وبه کار خودت مشغول نمیشی؟پیرمردمورف، اینو ببر به دنده دوآفرین... حالا سهاول کلاجو فشار بده، گیج!آروم رها کن -!خفه تام -چیکار کردی مورف؟اون کاری نکردلاستیک پنچر شدقانون مورفی -!خفه شو -بریم یدک رو برداریماین یدک بودبرو ابزار رو بیاراینجا قراره چطوری تعمیرش کنم؟مجبوری خودت فکر کنی. من همیشهاینجا نیستم که بهت کمک کنمچی شده، مورف؟چرا تو و مامان اسم بدیبرام انتخاب کردین؟اسم بدی انتخاب نکردیمقانون مورفی؟قانون مورفی این نیست کهقراره چیز بدی اتفاق بیوفته...معنیش اینه که اگه اتفاقی بیوفته،اتفاق خواهد افتادو این اسم برای ما موردی نداشتسوار شوسوار شو، بریمتایر پنچر چی؟ایول،این یه بدونِ سرنشین، ماله نیروی هوایی هندهسلول‌های خورشیدی این میتونه به کل مزرعه برق بدهفرمون رو بگیر، تام!برو برو بروگمش نکنیسریع‌تر تام، از چنگمون فرار میکنهخودشهدرست به اونجا نشونه بگیرنگهش‌دار، ایناهاش، میشهعالیه تامبابا؟کم مونده، ادامه بده، ادامه بدهبابا؟ -!تام -چیه؟ خودت میگی ادامه بدمفکر کنم این میشه همون قضیه"اگه بگم از پرتگاه بپر می‌پری"گمش کردیمنخیرممیخوای بچرخونیش؟بذار فرود بیاریم اونجا، درست لبه آبآفرینبنظرت چند وقته اون بالا بوده؟ماموریت کنترل دهلی پیش مثل ما با شکست مواجه شدیعنی چرا اینقدر آروم پایین اومد؟نمیدونمشاید خورشید مغزشو پُختهیا شایدم دنبال چیز دیگه‌ایهچی؟اون چاقو پهن رو بده منشاید دنبال یه‌جور سیگنالهنمیدونممیخوای باهاش چیکار کنی؟باهاش یه کار کاملاً اجتماعی میکنممثلاً تراکتور رو راه‌اندازی میکنمنمیشه بیخیالش بشیم؟به کسی آسیب نمی‌رسوندببین، این چیزا باید یاد بگیرن که چطورسازش پیدا کنن، مورف، مثل بقیه ماخب این چه‌جور کار میکنه؟شماها چیزی ازش فهمیدن؟کلاس دارماین یکی، باید صبر کنهچیکار کردی؟وقتی رفتی بهت می‌گنقراره عصبانی بشم؟نه با منخواهش میکنم، سعی کن نشینگران نباش، عصبانی نمیشمدیر اومدی، کوپبله، تایر پنچر شده بودو فکر کنم مجور شدی یه سری به مغازههواپیمایی آسیایی بزنیخیر قربان، راستش اون یهبدونِ‌سرنشین نظارتی بودبه همراه سلول‌هایخورشیدی فوق‌العاده. ساخت هندبشینیدخب، نمرات تام رو بررسی کردیمکشاورز خیلی خوبی میشهآره استعدادشو دارهدانشکده چی؟خب دانشگاه فقط نخبه‌ها رو قبول میکنهاونا واقعاً منابع زیادی ندارنمن هنوز هم مالیاتم رو پرداخت میکنم...خباین پول‌ها به کجا میره؟دیگه ارتشی نموندهخب، اونا به دانشگاه نمیرنببین، کوپ. تو باید واقع‌بین باشیمیخوای مانع دانشگاه رفتن پسرم بشی؟بچه نمرات تام اونقدرها هم بالا نیستدور کمرت چنده؟ساز منظورت رو نمی‌فهممیعنی بهم میگی سنجیدن باسنت دو رقم میبرهولی واسه سنجیدن آینده پسرم تنها یه رقم میبره؟ببین، تو آدم تحصیل کرده‌ای هستی، کوپو خلبان آموزش دیدهو مهندسباشه ولی الان به مهندسین نیاز نداریم،ما کمبود برق و حمل و نقل نداریمما کمبود غذا داریم،دنیا به کشاورزها نیاز داره، کشاورزهای خوبمثل تو، مثل تامکشاورزهای تحصیل کردهما مراقب نسل آینده هستیم، کوپو اوضاع داره بهتر میشه...شاید نوه‌هات بتوننتموم شد، قربان؟نخیر. خانم هنلی اینجاست که راجبمورف باهات حرف بزنهمورف بچه‌ی فوق ‌العاده‌ایه، خیلی باهوشهولی اخیراً کمی مشکل دارهاینو آورده بود به سایر دانش‌آموزا نشون بده"بخش مربوط به "فرود به ماهآره، یکی کتاب کارهای قدیمی‌امعکس‌هاشو خیلی دوست دارهاین یه کتاب‌کار فدرالی قدیمیهما ویرایشات جدیدش رو جایگزین کردیمویرایش شده؟بهشون توضیح میدیم که ماموریت‌های آپولو...چگونه‌ با کارهای جعلی‌اشاتحادیه شوروی رو ورشکسته کردباور نمی‌کنید به ماه رفتیم؟باور میکنم که این یه‌جور شعار هوشمندانه‌ای بود...که شوروی خودشو ورشکسته کردمنابع‌شون رو صرف موشک‌ها و سایرماشین‌های بدرد نخور کردنماشین‌های بدرد نخور؟اگه قراره بخواهیم ول‌خرجی‌های بیش از حدقرن باید به بچه‌هامون چیزایی درمورداین سیاره یاد بدیم. نه ترک کردن این سیارهمیدونید یکی از اون ماشین‌هایی که شما بهشبود MRI بدردنخور گفتین،و اگه الان از اونا باقی‌مونده بود...دکترها میتونستنتومور همسرم رو قبل از اینکه جان بده بفهمندو بجای این که من اینجا باشم، با هم اینجامی‌نشستیم و به حرفاتون گوش میدادیمو این خیلی خوب میشد چونهمسرم همیشه از من آرومتر بوددرمورد همسرتون متاسفم، آقای کوپرولی مورف درمورد مزخرفات آپولو با...هم‌کلاسی‌هاشدعوا کردواسه همین فکر کردیم بهتره شما رو از اینجریان باخبر کنیم و نظرات شما رو بشنویمکه در آینده بیشتر مراقب رفتارهاش باشهبله، میدونید چیه، فردا مسابقه هست،به فاز بیسبال علاقه‌مند شدهتیم مورد علاقه‌اش قراره بازی کنه...اونجا شکلات و نوشابه‌ هم می‌فروشنفکر کنم بهتره ببرمش اونجاچطور پیش رفت؟موقتی اخراج شدیچـــــی؟کوپرکوپر هستم، بلهکوپ، تراکتورهایی که فعال کردی قاطی کردنکنترل‌ها رو دوباره تنظیم کنکردم، ولی بهتره یه نگاهی بندازییکی پس از دیگری مزرعه رو خراب کردنیه چیزی باعث اختلال تراکتورها شدهمغناطیس. یا همچین چیزیمهم نیست کدوم کتاب افتاده. روش کار کردمخودت گفته بودیجاهای خالی رو شمردمچرا؟فکر کردم شاید داره با من ارتباط برقرار میکنهاز کُد مورس استفاده کردممورس؟(کد رمزی جهت برقراری ارتباط)...آره، نقطه‌ها و خط‌ تیره‌ها. اینمیدونم کُد مورس چیه، مورف...ولی هنوز فکر نمیکنمقفسه کتاب بخواد باهات حرف بزنهرو  GPS مجبور شدم قطب نماها ودوباره تنظیم کنم که با ناهنجاری منحرف نشندلیل؟نمیدونم...اگه خونه از آهن مغناطیسی ساخته شده بودما اینو فوراً با روشن کردن تراکتور می‌فهمیدیمشنیدم جلسه اولیا اونقدرا هم خوب نبودشنیدی؟ انگار مثل اینه که ماخودمونو فراموش کردیم، دونالدکاشفان، پیشگامان، نه مستخدمانزمانی که بچه بودم، احساس میکردمهر روز دارن چیز جدیدی اختراع می‌کننایده یا ابزارهر روز شبیه کریسمس بودولی فکرشو بکنو هر تک تکشون همه چی رو میخواستناین دنیا اونقدرا هم بد نیستو تام کارشو خوب بلدهتویی که متعلق به اینجا نیستیچه دخترم می‌دونست. خدا رحمتش کنهبچه‌هات میدونن، بخصوص مورفما عادت داشتیم به آسمون نگاه کنیم و...به خودمون بگیمکه مکان‌مون یه جایی میان ستاره‌هاست،حالا به پایین نگاه می‌کنیمو نگران مکان‌مون هستیم که در کثافت غرق شدهکوپر، تو در یه چیزی مهارت داریو فرصتشو پیدا نکردی که باهاش کاری بکنیمتاسفم...انتظار نداشتی که گرد و غبار بهت غذای آلوده بدهو تو رو مریض کنه و نابودت کنه،اگه اشتباه نکنم ماه آپریل،فکر کنم ساعت زمون من چه بازیکان‌هایی داشتیماین بچه‌ننه‌ها کی‌ان‌؟خب، زمان من، مردم گرفتار آب و غذا هستنسر غذا باهم مجادله می‌کنن، حتی بیسبالتو بیسبال پوفیلا نمی‌خورنمن هات‌داگ می‌خواممدرسه گفته قراره شغل منو ادامه بدیفکر کنم این عالیه -فکر میکنی عالیه؟ -بابا تو از کشاورزی متنفریبابابزرگ گفتپس بابابزرگ گفت؟ ببین، مهم اینه کهدرمورد این کار چه حسی داشته باشیمن که خوشم میاداز مزرعه‌مون خوشم میادو تو استاد این کار میشیبریمخیلی‌خب این یکی نابهدار و دسته، ماسک‌ها رو بزنینتام، مورف؟زدیممورف، تام، پنجره‌ها رو چک کردین؟مورفروحبالشت‌ رو بردارمیری با تام بخوابیروح نیستگرانشهتام رو برمیدارم، بعد میرم شهرمیخوای اینجا رو تمیز کنی؟بعد از اینکه دعاها تو خوندیاین مورس نیست، باینری‌یهکلفت یعنی نازک یعنی مختصاتخودشهنمی‌تونم از دست بدمبابابزرگ چند ساعت بعد برمیگرده، مورفولی نمیدونی قراره چی پیدا کنیمو بخاطر همین نمیتونی بیایمورف؟بابابزرگ چند ساعت بعد میاد، بهش بگواز بی‌سیم باهاش حرف میزنم!خدایاچیکار داری میکنی؟فکر میکنی خنده داره؟اگه من نبودم الان اینجا نبودیحالا یکم به درد بخورمورفمورففکر کنم اینجا آخر جاده هستسیم ‌بُر نیاورده بودی؟دختر خودمه!از حصار فاصله بگیر -نزن، نزن، غیرمسلحم -دخترم توی ماشینهنترساینجا رو چطور پیدا کردی؟دخترم کجاست؟مختصات این مکان در نقشه‌ات خط خورده بوداین مختصات رو از کجا پیدا کردی؟دخترم کجاست؟!مجبورم نکن دهنتو سرویس کنم، بتمرگفکر میکنی ارتشی هستی، رفیق؟دیگه نیروی ارتش وجود ندارهقراضه‌هایی مثل تو رو دارم کهعلف‌هامو باهاش می‌بُرماون مختصات رو از کجا پیدا کردی؟مگر اینکه دوست داری علف‌بُر دومم بشینه فکر کنم بهتره تو رو به خشک‌کن یاجارو برقی تبدیلت کنمنه نمیکنیتارس، لطفا برگرد عقباستفاده از برنامه امنیتی ارتش سابقکار خطرناکیهاونا قدیمی هستن و طرز تفکرشون نامعلومهبخاطر همین از طرف دولت عفو می‌شنکی هستی؟دکتر برندآره، یه دکتر برندی می‌شناختم، پروفسور بودچی شد فکر کنی من نیستم؟چون مثل شما خوشگل نبود،خواهش میکنم دکتر برنداصلاً نمیدونم اینجا کجاستبابت دخترم میترسم، و میخوام برم پیششاینکارو بکنید و من هرچیزی که میدونم رو بهتون میگماین و دخترش رو ببر به اتاق کنفرانسدخترتون خوبه. دختر باهوشیهجای تعحب نیست که مادر باهوشی دارهکاملاً معلومه که دوست نداریمراجعه کننده‌ای داشته باشیچرا نمیذاری سرمونو بندازیم پایین و برگردیم سر کارمون؟به این آسونیا نیستالبته که هستبنده هیچی راجع‌به تو نمیدونمهیچی از این مکان نمیدونمبله میدونی!باباسلام کوپرپروفسور برندبگو ببینم چطور اینجا رو پیدا کردی...تصادفی اینجا رو پیدا کردیم. ما عجله داشتیمشما در فوق محرمانه‌ترین مکان دنیا نشستینهیچکس نمیتونه بیاد داخلو نمیتونه بره بیرونکوپر، خواهش میکنم با این آدما همکاری کن...خبتوضیح دادنش یکم سختهما، ما این مختصات رو به واسطه نوعیناهنجاری پیدا کردیمچه‌جور ناهنجاری؟مطمئن نیستم به این بگم ماوراء طبیعیولی مطمئنم که علمی نیستباید کاملاً دقیق و بی‌پرده حرف بزنید، آقای کوپرهمین حالاگرانش بودچه‌جور گرانشی، این کجا بود؟...واقعاً خوشحالم که بابت گرانش خوشحال شدی، اماتا وقتی بهمون اطمینان‌خاطر ندینچیزی بهتون نمی‌گیماطمینان‌خاطر؟ -بله -مثلاً ما رو از اینجا ببرین بیرونمنظورم این نیست که ببرین توی ماشیننمیدونی ما کی هستیم، کوپ؟نخیر پروفسور، نمیدونمتو که پدرم پروفسور برند رو می‌شناسیما ناسا هستیمناسا -ناسا -همون ناسایی که ماموریت داشتیشنیده بودم شماها رو بخاطر رد اقدامِ...پرتاب موشک از لایه استراتوسفر برایکشتن آدمای قحط زده مهر و موم کردن...وقتی فهمیدن که کشتن انسان‌هاراه‌حل دراز مدتی برای این مسئله نیستما رو محرمانه برگردوندنچرا محرمانه؟چون نظرات عمومی مانع از کاوش در فضا بودنه تا وقتی که برایبدست آوردن غذا رنج می‌کشیهفت سال پیش گندم‌ با آفت نابود شدامسال هم بامیه نابود شد(گیاه گرمسیری)الان فقط ذرت موندهو ما بیشتر از پیش ذرت پرورش میدیمبله، مثل سیب‌زمینی‌ها در ایرلند و...طوفان گرد و غباری که در راههذرت بزودی از بین میره،ما راهی پیدا میکنم، پروفسورهمیشه راه‌حلی پیدا میشهبا این امید که سیاره زمین متعلق به ماست؟ما صاحب زمین نیستیم، نهولی خونه‌ی ماست‏‏ما حتی نیتروژن رو استنشاق نمی‌کنیم،ولی آفت‌ها نیتروژن رو مصرف میکننوقتی اینکارو میکنن، اکسیژن هوای ما کمتر میشهآخرین نسلی که قحطی میکشهاولین نسلی خواهد بود که خفه خواهد شد...نسل دخترتآخرین بازماندگان زمین خواهند بودمورف یکم خسته شده اگه اجازه بدیندوست داره کمی تو اتاقم بخوابهبله، ممنون میشمباشهحالا میخوای بهم بگی چه نقشه‌ایواسه نجات انسان‌ها داری؟قرار نیست دنیا رو نجات بدیمبلکه میخوایم ترکش کنیمگردشگرآخرین سفینه پیشرفته در کره‌ی خاکیپروژه اندورنسآخرین ماموریت ماشما آدم‌ها رو فرستادین بالا که خونه جدید پیدا کنن؟ماموریت‌های ایلعازراسم امیدوارکننده‌ای واسه ماموریتهایلعازر توسط عیسی زنده شدالبته، اون باید همون اول می‌مُردتوی منظومه‌ شمسيِ ما سیاره‌ای نیست که،توش امکان حیات باشهو نزدیک‌ترین ستاره هزاران سال از اینجا فاصله دارهاین ماموریت‌ها بیهوده محضهاون بیچاره‌ها رو کجا فرستادی؟...کوپرتا وقتی که خلبانيِ این ماموریت روبر عهده‌ نگیری چیزی نمی‌تونم بهت بگمتو بهترین خلبانی هستی که ما داریم -من از لایه استراتوسفر بیشتر پا نذاشتم -این تیم کاری با فرستادن شبیه‌ساز ندارهما به خلبان نیاز داریم، و این ماموریت‌ایهکه تو براش آموزش دیدیچی، بدون اینکه چیزی بدونم؟یه ساعت پیش اصلاً نمیدونستی بندهزنده هستم یا مُرده، و الان دست از سرم برنمیداریما چاره‌ای نداریمولی یه چیزی تو رو آورد اینجااونا تو رو انتخاب کردناونا" کی هستن؟"رفتنم چقدر طول میکشه؟ -گفتنش سخته -...سال‌هاپروفسور، من فرزند دارمبرو بالا، و نجاتشون بدهاونا" کی هستن؟"ما شروع به شناسایی کردیمبیشتر ایستگاه‌های فضایی در لایه‌های بالاتردچار انحرافات مختصری شده بودنددر واقع فکر کنم شما خودتون با این مشکل مواجه شدینبله، در تنگنای دریاسیستم کنترل پرواز هواگرد خراب شد و سقوط کردمدقیقا...ولی با وجود تمام این ناهنجاری‌هامهم‌ترینش این یکیه،در نزدیکی زحلاختلال فضا-زمان به‌وجود اومدهکرم چاله هست؟ -این و به کجا ختم میشه؟ -به کهکشان دیگر -یه کرم چاله همین‌جوری باناهنجاری ظاهر نمیشهیکی کرم‌چاله رو گذاشته اونجااونا"؟"و اونا هرکی هستن، بنظر میاد کهمیخوان ما رو پیدا کننکرم‌ چاله به ما این امکان رو میدهکه به سایر کهکشان‌ها سفر کنیمدرست موقعی که نیاز داشتیم ظاهر شددنیاهایی که پتانسیل زندگی در آنها هستو اونا این رو در معرض دید ما قرار دادن‏‏‏‏کاوشگران فضایی‌مانشما کاوشگران فضایی فرستادین اونجا؟ما ماموریت ایلعازر‏‏‏‏با احتمال وجود حیاتکاوشگرانی که توش انسان‌هایبسیار شجاعی رو حمل میکردنبه رهبری دکتر منمحفظه‌‌ی هرکدوم توانایی پشتیبانی زندگیبه مدت ولی میتونن با خواب ‌دراز مدتبازه‌ی زمانی زندگی رو گسترش بدنتا این که بتونن ده‌ها سال یا بیشترمطالعات در اون سیاره‌ها رو انجام بدنماموریت اون‌ها ارزیابی...سیاره مربوط به خودشون بود واگه امکان حیات در یکی از اونا...وجود داشته باشهبا ارسال سیگنال به ما...دوباره به خواب‌زمستانی میرونتا منتظر نجات بموننو اگه امکان حیات در سیاره‌ها نباشه چی؟با شجاعت تمام به دیار باقی میرنشما منابع کافی برای رفتن بهسراغ نهانتقال داده از داخل کرم‌چاله ابتدایی‌یه...پینگ‌های باینری [به ما سرنخ‌‌هایی در رابطه با سیاره‌ها رو میدهکه کدوم پتانسیل حیات داره. یکی این پتانسیل رو دارهاون یکی هم با این همه فاصلهمعلوم نیست بشه یا نشهیک منظومه با سه سیاره‌ی احتمال حیات؟احتمالش کم نیستبسیار عالی...اگه خونه‌ی جدید پیدا کردیمبعد چی؟احتمال این کمهداریم  B و یه نقشه  A یک نقشهآیا متوجه شدی یه چیزایی دربخشِ پرتاب عجیبه؟کلِ زیرزمین خودش یه دستگاه مرکز گریزهمثل یه وسیله نقلیه؟ ایستگاه فضایی؟اینه  A هر دو. نقشه -چطور میخوای اینو از زمین ببری بالا؟ -اولین ناهنجاری‌های گرانشیهمه چی رو عوض کردبعدش فهمیدیم که مهار کردن گرانش امکان‌پذیرهواسه همین رو یه تئوری کار کردم وشروع کردیم به ساختن این ایستگاهولی هنوز به نتیجه نرسیدیهست  B واسه همین نقشهمشکل خودِ گرانشهچطور بتونیم انبوهی از بشریت زیست‌پذیررو از سیاره ببریم بیرون؟B این تنها راهشه. نقشهانفجار جمعیتبیش از سنجیده شده با جِرم کمتر از چطوری اینا رو پرورش میدی؟با ابزارهای پیشرفته ما بعدش، با رحم اجاره‌ایبازه‌ی رشد سریع‌تر میشهدر عرضِ مشکل اساسی در استعمارتنوع ژنتیکی هستاین مشکل رو حل میکنهآره، ولی آدمای اینجا چی میشن؟میخوای همین‌جوری بیخیال جماعت بشیم؟بچه‌های من؟باحال‌تره  A واسه همین نقشهتا چه حد پیش رفتی؟ -تقریباً تمومه -...ازم میخوای که تمام دار و ندارم رویتقریباً" شرط‌بندی کنم؟" -ازت میخوام که بهم اعتماد کنی -خونه جدیدی برا ما پیدا کن...و وقتی برگشتی، من مشکل گرانش روبرطرف میکنم. بهت قول میدم!برومورف!برو. اگه میخوای ترک کنی، برواین دنیا برات کافی نیست، مگه نه کوپر؟چی، یعنی وقتی بخوام برم اونجادلیل میشه احساس کنم که ذاتاً این کاره‌ام؟یعنی منو شگفت زده میکنه؟نه اصلاًم اینطوری نیستشایدم اینطورهبه چیزی که درسته بابت نتیجه‌یاشتباهش اعتماد نکنبنیان همه چیز "چرا" هستو بنیان استوار و لمس کردنیه...ما کشاورزها هر سال اینجا می‌نشینیم"وقتی بارون می‌باره، میگیم: "سال بعدخب، "سال بعد" ما رو نجات نمیدهحتی سال‌های بعد از اوناین دنیا مث گنجینه هست، دونالد، ولی خیلیوقته به ما میگه اینجا رو ترک کنیمآدمیزاد در این سیاره متولد شدهولی قرار نیست اینجا بمیرهتام مشکلی نداره، ولی باید مورف رو متقاعد کنیمیکنمقول میدی ولی به قولت عمل نمیکنیباید باهام حرف بزنی، مورفباید اینو قبل از رفتنم درست کنمپس منم شکسته‌ی اینو نگه‌میدارم که تو بمونی،وقتی شما دوتا متولد شدینمامانتون چیزی بهم گفتکه منظورشو خوب نفهمیدم"...گفت، "حالا، ما اینجاییم که"خاطراتی برای بچه‌هامون باشیم"حالا می‌فهمم منظورش چیه،وقتی پدر مادر میشیشبحِ آینده‌ی بچه‌هات میشیگفتی که ارواح وجود ندارندرسته مورفنگام کنالان نمیتونم روح تو باشمباید وجود داشته باشماونا منو انتخاب کردن،مورف، اونا انتخابم کردن. خودت هم دیدیتو بودی که منو پیش اونا راهنمایی کردی!واسه همینه که نباید بری،پیام روح رو فهمیدمیک کلمه. میدونی چیه؟مورف -"بمان" -میگه بمون، بابا -مورف -باورم نمیکنی. به جای خالی کتاب‌ها"نگاه کن. میگه "بمان!چرا گوش نمیدی؟ میگه بمونمن برمیگردمکِی؟یکی برای تویکی برای من،وقتی اون بالا به خواب دراز مدت رفتم،یا با سرعت نور مسافرت کردم،یا نزدیک سیاه‌چاله شدمزمان برای من فرق خواهد کردو نسبت به من کندتر خواهد گذشت،پس، وقتی برگشتماینا رو باهم مقایسه میکنیم -زمان‌های ما متفاوته؟ -آرهشاید وقتی برگشتم، تو و من در یه سن باشیم!تو و من، یه سن، چی؟فکرشو بکنمورفتو نمیدونی کِی برمیگردیاصلاً نمیدونی -مورف -،مورف، این کارو نکن!نذار اینجوری رهات کنم. مورف!نذار اینجوری رهات کنم. مورفدوسِت دارممیشنوی؟همیشه دوسِت دارمو حتماً برمیگردمبرمیگردمچطور بود؟خوب بود، خوب بودبه سلامت، باباآره. مراقب خونه‌مون باش، خب؟میتونم از وانتت استفاده کنم؟ -منظورت وانت تو؟ -مطمئن میشم که اونو برگردوننمراقب بچه‌هام باش دونالدآماده روشن کردن موتورهای اصلیبا شمار معکوس!بابا! بابا!باباموتورهای اصلی آماده اشتعالسهدویکشروع اشتعالتمام موتورها خوبنآماده‌ی جداسازیآماده جهت جداسازی بخش اولبخش اولمارک رفقای من رو به راهن؟می‌بینم کلی برده واسه مستعمره آوردینبهش تنظیمات شوخ‌طبع دادن که باافرادش رابطه‌ی خوبی داشته باشهفکر میکنه ما رو آروم می‌کنهیه رباط کمدینچه ایده جالبیمن واسه گفتن جوک چراغ اشاره‌گر دارمخواستی میتونم از اون استفاده کنمنه، اشکالی نداره،آره، بعد از اینکه هوابند رو منفجر کردممیتونی از این چراغ واسه پیدا کردنخونه‌ات استفاده کنیدرجه شوخ طبعی‌ات چنده، تارس؟%اونو بکن جداسازی بخش دومتنظیم به حالت دستیتنظیم دستیغیرفعال سازی هیتر چک -کنترل بر عهده من -پاس به تو -چک.ABFسلول‌های سوخت موشک ترمزی -مختصرسخته همه چی رو پشت‌ سر بذاری...بچه‌هام، پدرتقراره کلی باهم باشیم -بهتره یاد بگیریم معاشرت کنیم -و بعضی وقتا هم نهخواستم صادق باشمخب، لازم نیست اونقدرم صادق باشیتارس، درجه صداقتت چنده؟،صداقت کامل از نظر دیپلماتیک و امنیتبرای ایجاد ارتباط باگونه‌های حساس مناسب نیستبسیار عالی‏‏خیلی‌خب، کنترل رو بدست میگیرمبه درگاه ماژول نزدیک می‌شیم، همش با تو، دویلآروم و تمیز، دویل. آروم و تمیزما رو ببر خونهقفل شدقفل شد -خوب بود -خب، کلاها به سرخوب بوددیر شارژ نمیشه. مهم نیستکوپر، الان باید کنترل رو داشته باشیدارمش -ارتباط با ماژول حلقه‌ای فعال شدشروع شدآماده‌ی دوران هستیم؟ -یه لحظه -سلام کیس -سلام تارس -خب، همه چی ردیفه -خیلی‌خب، بیایید شروع کنیم -‏‏(نیروی گرانش در سطح استاندارد دریا)گرانش اذیتتون نمی‌کنه؟راملی، حالت خوبه؟بله -خوبی؟ -آره، یکم وقت نیاز دارمخیلی‌خب، فکر کنم پشت محفظه چندتادیمن‌هیدرنات هست، میرم بیارممیرم اتاق تخت‌ها، چند ثانیه بعد میامبرند؟بله؟ -یکم زیاد بیار -آملیا، مراقب باشسلامم رو به دکتر من برسون -حتما -،گذرگاهتون خیلی خوبه، ما محاسبه کردیمدو سال بعد به زحل می‌رسینکلی دیمن‌هیدرنات باید بخوریمقربان، مراقب خانوادم می‌شین، درسته؟وقتی برگشتی، منتظرت خواهیم بود،پیرتر شدم، باتجربه‌تر شدمولی از دیدنت خوشحالم"مرگِ آسان بر پیرمرد، مرگی شایسته نیست"چه خواهیم و چه نخواهیم""مرگ سراغ ما خواهد آمد"تا آخرین لحظاتِ سال‌خورده‌ات مقابله کن"،عاقلان می‌دانند""مرگ جزئی از سرنوشت استفایده‌ای ندارد قدرت بر آنی که""عمر ازش گذشته است"مرگِ آسان بر پیرمرد، مرگی شایسته نیست""تا آخرین لحظاتِ سال‌خورده‌ات مقابله کن"(شعری از دیلن تامس شاعر اهل ولز)بذار فشارتو بسنجمخیلی تنهانما همدیگه رو داریموضع دکتر من از ما بدترهمنظورم اوناستسیاره‌ بی‌نقضیه و ما هیچ‌وقت کره‌ایمثل اونو پیدا نخواهیم کردنه، شبیه متجمع نیست که بریم دنبالشنژاد انسان‌ قراره سرگردان بشهنومیدانه دنبال سنگی هستن که محکماونو بگیرن درحالی که نفس‌شون رو گرفتن،باید اون سنگ رو پیدا کنیمجایی که این سه سیاره با احتمالپتانسیل حیات رو پیدا خواهیم کرد،اولی سیاره "میلر" هست، لورابرنامه بیولوژیکی‌مون رو شروع کرده بودسیاره "ولف اِدموند" هم اینجاست -راجع‌به اِدموند بهم بگو -ولف" متخصص فیزیک اتمی بود"اینا خانواده داشتن؟نه، خانواده‌ای ندارن، پدرم در این تاکید میکردهمه اونا احتمال میدادن که بتونندوباره انسان‌ها رو ببینندامیدوارم که بتونیم اقلاًسه‌تای اونا رو خوشحال کنیمدکتر من چی؟دکتر من، خب، اون فوق‌العادستاون بهترینه ماستاون باعث الهامه در این سفر تاریخی و تنها شددانشمندان، کاشفانبخاطر همینه که دوست دارممیدونی، اونجا ممکنه بااتفاقات عجیبی مواجه بشیم، مثل مرگولی نه شرارتفکر نمیکنی طبیعت میتونه خبیث باشه؟ -نه. عظیم، خوفناک -ولی خبیث نهآیا شیر تیکه پاره کردن غزال رواز روی شرارت انجام میده؟فقط اونایی که پیش خود داریم رو می‌بریمبلهاین تیم نشانگر ویژگی بارز انسانیته -حتی من؟ -میدونی دیگه، به بیدار نمون -چند دقیقه بعد میرم خواب -یادت باشه، کوپداری الکی نفس‌تو به هدر میدیتارس، بیا گذرگاه رو یه ‌بار دیگه بررسی کنیم،هشت ماه تا مریخ،شتاب‌گرانشی در اطراف،‏‏‏‏مختصات کاملاً درستهبذار یه چیزی ازت بپرسمدکتر برند و اِدموندچرا پچ پچ میکنی؟اونا نمیتونن صداتو بشنوندکتر برند و اِدموند، خیلی صمیمی بودن؟ -نمیدونم -‏‏من تنظیم "قدرت تصمیم‌گیری" دارماما تنظیم "قیافه عاری از بیان" نداری، قراضهسلام بچه‌هاباباتون قراره یه مدت طولانی بخوابهخواستم فقط یه اطلاعی بهتون برسونمتماشای زمین از اینجا حیرت‌انگیزهگرد و غباری دیده نمیشهواقعاً امیدوارم که کارتون روبه نحو احسن انجام میدین،میدونم این پیام رو دریافت خواهین کردپروفسور برند قول داد که این پیام روبهتون میرسونه. خیلی‌خب، دوستتون دارماونه؟ -فکر نمیکنم، مورف -شما باید "دونالد" باشیدسلام مورف -چرا سوار وانت پدرم شدی؟ -ازم میخواست که اینو به برادرت بیارمبهت یه پیامی فرستادهبابت رفتنش خیلی ناراحت شده،اگه پیغامی اینجا ضبط کنینبنده میتونم اینارو مخابره کنم به کوپرمورف، دختر باهوشیهشاید من اونو عصبانی‌تر از قبل کردماون همین الانم معلم‌هاش رو مچاله کرده...خب، شایدمبخواد شما رو هم مچاله کنهالان کجان؟ -میرن به سمت مریخ -،دفعه بعد که از کوپر خبر شنیدیاونا به زحل خواهند رسیداونا ازم خواستن دوباره آسیب‌شناشیگیاهان رو بخونمخیلی مزخرفه. ولی گفتن که میتونمکشاورزی پیشرفته رو یه سال زودتر شروع کنمپس... خیلی‌خبخب، باید برم، فعلاامیدوارم جات اونجا امن باشهشرمنده کوپ، از مورف خواستم سلامی بهت بدهولی دست از لجبازی برنمیدارهدفعه بعد سعیمو میکنمروبه‌راهی، رام؟این فکر خیلی اذیتم میکنهاین، این،چندین میلی‌متر آلومینیوم، همین،اینا دیواری مابین مرگ و زندگیاونم در فضایی تاریک با میلیون‌ها مایل فاصلهمی‌دونستی برخی از بهترین...قایق‌ رانان انفرادی دنیا،شنا کردن رو بلد نیستن؟ اصلاً بلد نیستنو اگه جایی عمیق بیوفتن، کارشون تمومهما کاشفیم، راماین کشتیه ماست. بیااین کاوشگرِ مخابره هست؟ -دور مدار کرم چاله می‌چرخه -فرضاً این کرم‌چاله هست و هردفعه که...کاوشگر اینو دور میزنهما تصاویری از اونطرفکهکشان بیگانه رو دریافت می‌کنیممثل پرسیکوپ -دقیقاً -،پس این باعث میشه که بفهمیماونور قراره با چه چیزایی مواجه بشیم، درسته؟از منظر هدایت، بله،بچه‌ها، بنظر میاد تا سه ساعت دیگهبه کرم‌چاله نزدیک خواهیم شدکوپ، میشه چرخش رو متوقف کنیم؟چرا؟ -چون بقدر کافی نزدیکش شدیم -که اینطور!اوناهاش، خودشه، خودشه!کرم‌چاله اونجاستحرفتو یه راست بگو، راماون یه کُره هست،خب، البته که اینطورهولی از کجا میدونی چاله هم هست؟نه، منظورم، این همه توضیحات و...عکس‌هایی که دیده بودمتوضیحات بهت نشون میدن که اینچطور کار میکنهپس اونا بهت گفتن که باید از اینجابه اینجا بریاما مسیرش خیلی طولانیه...واسه همین کرم‌چاله فضا رو اینجوری خم میکنهیک میانبر به بُعدی بالاتر باز میکنهخیلی‌خب، پس نشون میده که...این فضای ،فضای ،که کرم‌چاله رو به ،یک دایرهاما دایره در یک کُره -دقیقاً. یه کره‌ چاله‌ای -ولی کی اینو گذاشته؟باید از کی تشکر کنیم؟نه، تا وقتی که سالم و سلامت ازش رد نشیماز کسی تشکر مشکر نمی‌کنم، رامواسه این ترفند خاصی هست، دویل؟کسی نمیدونهاونا" ساختنش، درسته؟"اقلاً برخی‌هاشونهمگی آماده‌این ازمنظومه شمسی‌مون خدافظی کنیم؟از کهکشانمونرفتیمکنترل‌ها اینجا کار نمی‌کننما از درونِ حجم عظیمی رد میشیمفضا-زمان فراتر از فقط میتونی مشاهده کنی و گزارش بدیاون چیه؟فکر کنم "اونا" هستن -اعوجاج فضا-زمان؟ -نکن، نکنچی بود؟دست فشاری اول...ماما رسیدیمارتباطات قطع شده به دستمون رسید -چطور؟ -فرستنده این طرف تونسته دریافت کنهاطلاعات پایه‌ای مربوط به سال‌ها کاوشسیگنال‌های "ملیر" و دکتر "من" جواب میدهماله اِدموند سه سال پیش قطع شد -شاید فرستنده‌اش خراب شده؟ -شاید. اونم مثل بقیه تا این موقعسیگنال می‌فرستادولی هنوزم ماله "میلر" خوبه، نه؟چون سریعتر از بقیه دریافت میشهاما مشکلی هست"سیاره نسبت به بقیه به "گارگنتواخیلی نزدیک‌ترهگارگنتوا؟(غول آسا)به سیاه‌چاله می‌گیمسیاره‌های میلر و دکتر مندورش می‌چرخنسیاره میلر روی "افق رویداد" ـه؟(جایی که برگشت از آن به بعد غیرممکن است)مثل توپ بسکتبالی که دور میله‌ می‌چرخهاگه بخوایم به سیاره فرود بیاییمخطرات خودشو خواهد داشت،و سیاه‌چاله‌ای مثل اونقدرت گرانش خیلی زیادی دارهباشه، من میتونم از اطرافِ...اون ستاره‌ی نوترونی دور بزنمنه، نه، مسئله این نیست. مسئله زمانه...بخاطر شدت گرانشزمان ما نسبت به سیاره زمینبشدت کاهش پیدا میکنهچقدر بد؟هر ساعت سپری شده در اون سیارهمعادل هفت ساعت در کره زمینهخدایا -به این میگن نسبیت -...خب، نمیشه یه جوری فرود بیاییم کهکوپرما یه ماموریت داریم -بله، دویل، ما یه ماموریت داریم -اینه که سیاره‌ای  ،A و ماموریت ما، نقشهپیدا کنیم که قابل سکنی باشهمثل همونی که آدما توی زمین زندگی میکنن،نمیتونی فقط به خانوادت فکر کنیباید به فراتر از این فکر کنی، خب؟من به خانواده خودم و میلیون‌هاخانواده دیگر فکر میکنم،میشه  A این تاخیر زمانی مانع نقشه...و وقتی کارمون تموم شدآدمای زمین تا اون وقت خواهند مُردنه، نمیشهداریم  B واسه همین نقشهخب. کوپر راست میگه، باید به زمان،مثل یه منبع نگاه کنیم، مثل اکسیژن و غذارفتن به اونجا، هزینه بردارهخیلی‌خب، ببین، داده‌ی دکتر من،امیدوار کننده هست، خب،اما ماه‌ها طول میکشه بریم اونجاو ماله اِدموند هم خیلی بیشترحالا میلر اطلاعات زیادی ندادهاما چیزایی که مخابره کرده، خیلی خوبهآب و مواد آلی هستبه این سادگی‌ها پیدا نمیشه...نه. نمیشه. و به منابعی مثل زمان فکر کنیدکه بخواهیم بعد از مدتی برگردیم اینجاراملی -بله؟ -چقدر فاصله به سیاره میلر نیاز داریمتا تحت تاثیر تداخل زمانی قرار نگیریم؟تا راس منحنی،که اینجاستدرست بیرونِ سیاره‌ی میلرباشهخیلی‌خباین گارگنتوا، اینم سیاره میلربجای اینکه اندورنس رو ببریم به مدار...سیاره‌ی میلر، که باعث،صرفه جویی در سوخت میشه،اما کلی زمان از دست میدیم،میتونیم یه مدار گسترده‌تر دور گارگنتوا انتخاب کنیم،با سیاره‌ی میلر هم راستا بشیم،خارج از تداخل زمانی، اینجا،بعد کاوشگر رو می‌فرستیم پایین،میلر و آزمایش‌هاش رو برمیداریم،برمیگردیم اینجا، اونا رو آنالیز میکنیممستقیم میریم، میاییم، یکم سوختبه خرج میدیم ولی کلی زمان ذخیره میکنیماین میشه -خوبه -اونجا دیگه وقت برا چت کردن والکی اینجا و اونجا رفتن نیست،واسه همین، تارستو باید حتماً اینجا بمونی،کیس، تو با منیکسی هست بخواد اینجا بمونه؟،اگه حرف از چند ساله،میتونم از زمان برای تحقیق درباره گرانش،و مشاهدات مربوط به کرم‌چاله استفاده کنماینا به پروفسور برند خیلی کمک میکنه،تارس، یه مدار مناسب دور گارگنتوا محاسبه کنسوخت رو ذخیره کن، شدت رانشگر رو کاهش بده،مطمئن‌شو در گستره سیاره میلر باشیمگرفتی؟من شما رو تنها نمیذارم، دکتر برندکیس، آماده‌ای؟ -بله -تو زیاد حرف نمیزنی، نه؟تارس به جای هردمون صحبت میکنهجدا شوراملی، میتونی نیروها رو برداشت کنی؟باور نکردنیهدر یک کلام، قلب تاریکی،اگه بتونیم ستاره‌ی متلاشی شده درون سیاه چاله،یعنی تکینگی رو ببینیممی‌تونستیم مشکل گرانش رو حل کنیمنمیشه این کارو کرد؟،هیچی از "افق رویداد" نمیتونه فرار کنه،حتی نور هم نمی‌تونه. همه جواب‌ها اونجاستولی به هیچ طریقی نمیشه دیداینم از سیاره میلرخدافظ رنجرورود اتمسفری برای این سریعه، نمیشه ازرانشگرها واسه کاهش سرعت استفاده کنیم؟،نه. میخوام از ایزودینامیک کاوشگر استفاده کنمکه کم سوخت مصرف کنیمترمز بادی؟ -مگه نمی‌خوایم سریع بریم پایین؟ -میخوایم سالم بریم پایینمحکم بشینینبرند، دویل، آماده باشینبه ترمز بادی نیاز خواهیم داشت -آماده‌باش، کیس -تنها زمانی که سقوط کرده بودم بخاطرموقعیت بد حرکت ماشین بودما رو به کشتن میدیفرض کن ماشین رو دیوانه‌وار می‌رونیخیلی تند میریم -هواشو دارم -آماده‌ی جداسازی بشم؟نه، میخوام هوا رو احساس کنمایناهاش!فقط آبـهمایه‌ حیات -موقعیت ثابت از فرستنده‌اش رو داری؟دارم، میتونی آرایش رو تغییر بدی؟میخوام سرعتو کاهش بدم. قراره یکم!انحراف داشته باشیم، محکم بچسبین‏‏‏با علامت من، کیسبا علامت من‏‏‏!حالاخیلی با وقارنه، ولی خیلی موثرمنتظر چی هستی؟برو، بروبرو، برو، برویک ساعت معادل هفت ساعتبشمریناز اینور‏‏‏جاذبه اذیت میکنهخیلی وقته توی سفینه شناور موندی؟‏‏‏یالا، یالا، یالااینجا هیچی نیست -باید اینجا باشه -...اگه، اگه سیگنال از اینجا اومده، پسفرستنده‌اشلاشه -بقیه‌اش کو؟ -اونور کوه هستاونا کوه نیستموج هستن -چی؟ -لعنتیکی به جلوی ما میادبه دستگاه ضبط نیاز داریمبرند، دویل همین حالا برگردین به رنجر!من بدونِ داده‌هاش جایی نمیرمفوراً برگرد. وقت نداریمموج دوم داره میادما وسط غلتاب قرار گرفتیم!پیدا کردم -!گمشو برگرد بیا اینجا -!برند، همین الان برگردکوپر، برو، کوپر برو!نمیتونم برسم. بروکیس، برو بگیرش!برو -بلندشو، بلندشو، برند -برو، برو، برو!نمیتونم -!میتونی، میتونی -!کیس گرفته. کیس گرفته!زود برگرد اینجا، دویلیالاببر به دریچهبرو، برو، برودریچه داخلی رو به حالت دستی تنظیم میکنم!کوپر نکن!کوپر نکن!کوپر!آب به موتورها درج کرد!دریچه رو باید ببندم!لعنتیکیس، چی شده؟ -مجرا پر شده از آب، گذاشتم خالی بشه -!لعنتی!گفتم که رهام کنی!و منم گفتم کونتو بکش بیار اینجاتفاوتش اینجاست که یکی از مابه ماموریت فکر میکنه!تو فقط میخوای بری خونه!میخواستم کار درست رو بکنممیتونی اینو به "دویل" بگی؟کیس، چقدر وقت تلف شد؟ -مایه حیات، ها؟این چقدر برامون بها داره، برند؟زیادده‌ها سالچه بلایی سر "میلر" اومده؟ -...باتوجه به لاشه‌ها -اون به محض فرود، توسط امواج از بین رفتهبعدِ این همه سال لاشه‌هاشچطور ممکنه کنار هم باقی بمونه؟بخاطر اُفت زمان،اگه از دید زمان به سیاره نگاه کنیم...اون چند ساعت پیش فرود اومده بود، اوناحتمالاً چند دقیقه پیش مُرده بودداده‌ای که "دویل" دریافت کرده بودحالت بی‌پایان پژواک امواج بودنه، آماده‌ی این نیستیمشما کله‌خرها استعدادهایزنده موندن رو خوب بلدین،خب، ما تا این مرحله کار اومدیمفراتر از هر انسان دیگه‌ای در تاریخ،اونقدرم فراتر نیست، حالا اینجا گیر کردیم...تا وقتی که دیگه کسیتوی زمین باقی ‌نمی‌مونهمنم مثل تو هر دقیقه رو می‌شمارمهیچ راه‌حلی نداره؟...چه بدونم، یه راهی، نمیدونمشاید بتونیم بریم دورن سیاه‌چاله وچندین سالِ از دست رفته رو پس بگیریم؟!سرتو اینجوری تکون نده -زمان نسبیه، خب -...میتونه گسترده بشه، متراکم بشه، ولینمیتونه برگرده عقب، نمیشهتنها چیزی که میتونه مثل زمان ازابعاد بگذره، گرانش هستباشه،گونه‌هایی که ما رو اینجا هدایت کردناونا میتونن از طریق گرانشارتباط برقرار کنن، درسته؟ممکنه از آینده با ما صحبت کرده باشن؟شاید...خیلی‌خب، اگه اونا میتونن -...اونا -گونه‌هایی بیگانه از فضای زمان، میتونه برای اونا شبیهبُعدی فیزیکی جلوه بدهگذشته، ممکنه برای اونا شبیه،غاری باشه که بتونن برن توش،و آینده، شبیه کوهیه که میتونن برن بالا!ولی برای ما اینطور نیست، خبببین کوپر، من گند زدم، متاسفمولی تو نسبیت رو خوب بلدیبرنددخترم نتونستم قبل از رفتنم بهش،تئوری انشتین رو یاد بدمنمیتونستی بهش بگی کهمیخوای دنیا رو نجات بدی؟نهوقتی صاحب فرزند میشییه چیزایی برات خوب روشن میشه،اینه که باید از امنیت بچه‌ات خاطرجمع بشیو این نمیذاره به یه دختر که دنیا قراره از بین برهکوپرزمان ایده‌آل برای موتورها چقدره؟ -یکی دو دقیقه -!خب، اینقدر وقت نداریم!کلاه‌ها به سربرند، تو کمک خلبانی. کیس، کومهاکسیژن رو از داخل رانشگرها فعال کن!قراره جرقه بزنیم -دریافت شد -تخلیه گاز!موتورها روشن شدسلام، رام -سال‌ها منتظرتون موندم -چند، چند سال؟الان میشه دویل؟...فکر میکردم آماده‌ام...تئوری رو میدونستمواقعیت فرق میکنهو میلر؟اینجا چیزی برای ما نیستچرا نخوابیدی؟ -یکی دوبار اینکارو کردم -دیگه فکر نمیکردم برگردینیه چیزایی در رابطه با هدر رفتن زندگیم اشتباه بودتا جایی که می‌تونستمچیزایی از سیاه چاله یاد گرفتمولی نتونستم چیزی به پدرت مخابره کنمما سیگنال‌ها رو دریافت می‌کنیماما نمی‌تونیم ارسال کنیمزنده هست؟آرهآره؟ -بله -پیام‌های چندین سال رو ذخیره کردیم -کوپر -مدت پیام‌ها از اول شروع کنسلام بابا، خواستمیه سری بزنم و سلام بدمترم دوم هم تموم کردمخانم چارلیز هنوزم امتیاز "ضعیف" میده،باعث شد عقب بیوفتمولی رو هم رفته خوب بودبابابزرگ به مراسم اومده بودبا یه دختر دیگه آشنا شدممن... فکر کنم عاشقش شدماسمش "لوئیز" هستاینجاستمورف، کامیون بابابزرگ رو دزدیدله و لورد کرد، حالش خوبهسلام بابانگاه کن!بابابزرگ شدیاسمش "جسی" ـه،میخواستم "کوپ" رو هم بیارمولی لوئیز گفت که دفعه بعدو دونالد نخواست اسم کاملتو بهش بدمگفت که خودش به قدر کافینقش مهمی داشتبا بابابزرگ خدافظی کنخدافظ بابابزرگشرمنده، خیلی وقته گذشته...این... همگی با همپدربزرگ هفته پیش فوت کرد...اونو پیش مامان و جسیدفن کردیمجایی که میخواستیم سنگ‌قبریبرات درست کنیم، اگه احیاناً برنگردیمورف هم تو مراسم خاکسپاری بوداونو زیاد ندیدیم، ولی اومده بودمیدونم که به این گوش نمیدی...همه این پیام‌ها فقط... بیهوده هستبیرون در تاریکیلوئیز گفت که دیگه بیخیالت بشم...و...پسگمونم... باید بیخیالت بشمنمیدونم کجایی، بابا...ولی امیدوارم در آرامش باشی، وخداحافظسلام، باباسلام مورف -!ای حرومزاده -از وقتی که میتونستی به اینا پاسخ بدیچیزی نگفته بودمچون بابت رفتنت خیلی عصبانی بودم،و وقتی خبری ازت نشداحساس کردم که دیگه باید با اینحس زندگی کنم، و اینطورهولی امروز تولدمه،و این یکی مهمه،چون بهم گفته بودی،زمانی بهم گفته بودی که وقتی برگردیممکنه سن هم باشیمو امروز به همون سنی هستمکه منو ترک کرده بودیپس الان وقت خوبیه که برگردیقصد مزاحمت نداشتمقبلاً تو رو اینجا ندیده بودمچون قبلاً هرگز اینجا نبودمهر دفعه با آملیا حرف میزنمخیلی کمک میکنهخوشحالم که تو هم شروع کردی -نه، دیگه نمیکنم -حرفی تو دلم مونده بود که خواستم خارجش کنماونا هنوزم اونجانمیدونمدلایل متعددی وجود داره که چرا ارتباطاتموننمیتونه رد و بدل بشهمیدونم پروفسورمطمئن نیستم که از چی بیشتر می‌ترسم...اینکه اونا اصلاً برنمیگردن یابرمیگردن و می‌بینن که ما مُردیمپس بیایید موفق بشیم -...پس -برگردیم به تکرار چهارمبذار اینا رو در زمینه‌های جدید اجرا کنیمبا تمام احترام، پروفسور، ما اینوصدها بار انجام دادیماین قراره تنها یک بار کار کنه، مورفهر پیچی که باهاش کار میکردنممکن بود گلوله به جاش باشه،ما اینجا، خیلی خوب پیش رفتیمچه بخواهیم چه نخواهیم، در تعادلرخنه کردیم، قبل از اینکه بمیرماینقدر به مرگ فکر نکنین، پروفسور -من از مرگ نمی‌ترسم -بنده یه فیزیکدان فرسوده‌اممن از زمان می‌ترسمزمانشما از زمان می‌ترسینسال‌ها سعی کردیم تعادل رو حل کنیم...بدون اینکه بخواهیمتغییری در ظاهر قوانین زمان بدیمو؟،یعنی، هر تکرار قصد داره حکم خودشو اثبات کنهاین رابطه بازگشتی‌یهاین بی‌معنیهتو به سال‌ها کارکردم میگی بی‌معنی، مورف؟نه، منظورم اینه که شما سعی کردین...این رو خودتون تنها تمومش کنینولی خیلی ساده به جوابشون رسیدیننمی‌فهمم چرامن دیگه پیر شدم، مورفمیشه اینو برا دفعه بعد بحث کنیم؟میخوام با دخترم حرف بزنموقتی پا به جهان گشودیمخودمون رو باید با واقعیتسفرهای میان‌ستاره‌ای مواجه کنیمباید به دنبال حیات‌هاییفراتر از حیات خودمان باشیم،باید بیندیشیم،نه فقط به خودبلکه به عنوان یک نژاد"مرگِ آسان بر پیرمرد، مرگی شایسته نیست"تارس، اندورنس رو درست جایی...قرار داد که بهش نیاز داشتیمولی سفر به سیاره بیشتر ازآنچه که فکر میکردیم زمان برددیگه سوخت کافی واسه رفتنبه هر دو سیاره‌ رو نداریمپس باید یکی رو انتخاب کنیمولی چطور؟هر دو احتمال پتانسیل رو دارن،و داده‌ی اِدموند بهترهولی ماله دکتر من...هنوز هم مخابره میشه. پسدلیلی نداریم که بگیم داده‌ی اِدموند خوب نیستدنیاش عناصر کلیدی برای زندگی دارهماله دکتر من هم همینطورکوپر، این بخش منهو واقعاً فکر میکنم ماله اِدموند جواب میدهچرا؟گارگنتوا، بخاطر همینیه نگاهی به سیاره میلر بندازهیدرو کربن، مواد آلی، بلهولی بدون زندگی. بیهودههمینا رو در "من" هم پیدا میکنیمبخاطر سیاه‌چاله؟قانون مورفیهر اتفاقی بیوفته، خواهد افتاد،تصادفات اولین آجرهای تکامله،ولی وقتی دور سیاه‌چاله می‌چرخی،زیاد از این اتفاقات نمی‌افته،ستار‌ه‌ها رو شهاب‌ها رو می‌مکهسایر اتفاقاتی که در غیر اینصورتممکنه سراغت میادباید از سیاه‌چاله فاصله بگیریمخودت گفته بودی دکتر منبهترینه ماستاون فوق‌العادستما فقط بخاطر اون اینجاییمو اونم الان اونجاستتوی سیاره هستو بهمون پیام خیلی آشکاری فرستادهکه بریم به سیاره‌اشدرستهولی اطلاعات اِدموندخیلی امیدوار کننده هستباید رای گیری کنیم،خب، اگه قراره رای گیری کنیم،چیزی هست که تو هم باید بدونیبرند، اونم حق داره بدونهاین ربطی نداره -چی؟ -اون عاشق "ولف اِدموند" ـه -این حقیقت داره؟ -آرهو این مجبورم میکنه که حرف دلم رو دنبال کنمشاید خیلی وقته اینجاییم وبا فهمیدن تئوری‌ها سرو کله میزنیمتو دانشمندی، برند -پس گوش بده -وقتی میگم عشق...چیزی نیست که ما ابداع کردیم، اینرویت پذیره، قدرتمندهاین باید مفهومی داشته باشه -...بله عشق مفهوم داره، ابزار اجتماعی -،وابستگی اجتماعی، پرورش انسان -،ما انسان‌هایی رو دوست داشتیم که مُردن -اینجا ابزار اجتماعی کجاست؟ -نیست -،شاید این مفهومی فراتر داره...چیزی که ما هنوزنمی‌تونیم درک کنیم...شاید یه مدرکه، یه،یه ساخته‌ای از بُعد فراترکه ما از روی قصد نمی‌تونیم درک کنیم...خودمو کشوندم به کهکشانی بیگانه کهدنبال کسی بگردم که ده‌ها سال صورتشو ندیدمکسی که میدونم، احتمالاً مُرده...عشق تنها چیزیه که ما قادریماون رو از میان ابعاد فضا-زماناحساس کنیمشاید بهتره به این اعتماد کنیمحتی اگه نتونیم مفهومشو درک کنیمخیلی‌خب، کوپردرستهحتی کوچک‌ترین احتمال اینکه بتونمولف رو دوباره ببینم، هیجان‌زدم میکنهولی این باعث نمیشه اشتباه میکنم -...راستشو بخوای -اشتباه میکنیتارس، مختصات دکتر من رو تنظیم کندوباره یک‌سوم از دست دادیمسال بعد. سال بعدمجبورم تو مزرعه نلسون کار کنمو باید اینکارو بکنمچه بلایی سر "نلسون" اومده؟مورف، چیز دیگه‌ای لازم داری؟بازم کیک‌شکالاتی میخوای؟نه، سیرم. ممنونخیلی خوشمزه بوددونات رو لطف میکنیمیشه شب رو بمونی اینجا؟اتاقت دست‌نخورده باقی‌موندهباید برگردم،چرخ‌ خیاطی‌ام اونجاست...اما کلی...منخاطرات زیادی اینجاستخب، شاید چیزی برا این داشته باشیمکوپغبار...لوئیز، من یه دوستی دارم که میتونهبه ریه‌هاش نگاه کنهآملیا، شرمنده -تصمیم بی‌طرفانه گرفتی -مگر اینکه میخوای منو بابتگنده‌کاری در سیاره میلر مجازات کنینه، این تصمیم شخصی نبودخب، اگه اشتباه کنی، به یه تصمیمکاملاً شخصی تبدیل میشهمحاسبات سوختتبر مبنای برگشت از سفرهبعد از اینکه کار با سیاره "من" تموم شد...مجبور میشی تصمیم بگیریکه برگردی خونه، یا بری به اِدموندB با عملی کردن نقشهشروع مستعمرهممکنه ما رو از انقراض نجات بدهامکان داره دوباره تصمیم بگیری...که آیا میخوای بچه‌هات رو ببینییا نژاد آدمیزاد رو از انقراض نجات بدیامیدوارم اونوقت هم تصمیم بی‌طرفانه بگیریاز وقتی که اومده بود شما رو میخواستسعی میکردیم پیداتون کنیممورف -سلام. اینجام پروفسور -فکر کنم همه رو ناامید کردمنه، ما رو تا این مرحله آوردینخیلی نزدیک شدیممن کار شما رو تموم میکنمخوبه، خوبه، مورفتو امید داری،تمام، تمام این مدتازت خواستم که امید داشته باشیازت میخواستم که باور کنی، پدرت برمیگردهباور میکنم، پروفسورمنو ببخش، مورف -چیزی برای بخشش نیست -دورغ گفتم، مورفبهت دروغ گفتمنیازی نبود که برگردههیچ راهی واسه نجات ما نیستچی؟  A ولی نقشههمه اینا؟تمام انسان‌ها؟ تعادل گرانش؟پدرم می‌دونست؟منو ترک کرد؟"...مرگِ... آسان... بر... پیرمرد"نه، نه!نمیتونی بری، نهدکتر برند، متاسفم که اینو براتون میگمپدرتون امروز فوت کردبا آرامش و بدون درد، درگذشتبهتون تسلیت میگمبرند، میدونستی؟بهت گفته بود، نه؟میدونستیهمش یه دروغ بودتو ما رو ترک کردیکه خفه بشیمقحطی بکشیممترجم: پوریا ضیامنشRemingtonابرهای منجمدچیزی نیستچیزی نیستچیزی نیست،بهتر از این خوشحالی نمیشهچقدر خوبه که آدم چهره‌ متفاوتی رو ببینهبعد از این همه مدت، امیدی برای این نداشتممنابعم به شدت کم شده بودن،آخرین باری که به خواب رفتمروش تاریخ بیداری نذاشتمشما منو از مرگ بیدار کردینماموریت ایلعازربقیه چی؟ -متاسفانه، فقط شما موندین -البته، تا اینجای کار...نه، با توجه به شرایط حاکمشانس خیلی کمی واسه نجات بقیه داریمدکتر مندکتر من، راجع‌به دنیاتون برامون بگین،دنیامون، البته امیدوارمدنیامون سرد، خشن، اما زیباستروزها به طول شب‌ها شرایط گرانش خیلی مناسبه،اینجا، جایی که فرود اومدمآب قلیایی‌یهو آمونیاکِ هوا برای اسنتشاقچند دقیقه‌ای خیلی زیاده،اما، زیر سطح، ایناهاش، پاییناینجا کلر پراکنده میشه، امونیاک همباعث کریستالی شدن هیدرو کربن‌ها میشهبرای گونه‌ها، قابل استنشاقهپتانسیل داره برای زندگیاحتمال زیاد میتونیم اینجا زندگی کنیماین مطالعات مربوط به سطحه؟سال‌هاست که کاوشگرهای گوناگونیبه مناطق مختلف فرستادمگسترده‌ی اکتشافتون چقدر پیشرفت داشته؟مدیریت خیلی از ماموریت‌های مهم...بر عهده من بود، ولی با محدودیتمنابع اکسیژن، "کیپ"، بیشترکارها رو بجای من انجام دادهمشکلش چیه قربان؟نابکاری. هویت گونه‌های اولیه‌ی...کریستا‌ل‌های آمونیاک روبه درستی شناسانی نکرد، مدتی بود که،شرایط سختی رو می‌گذروندیم،اما من اونو کاملاً از کار انداختماز منبع قدرتش برای ادامه ماموریت استفاده کردمقبل از اینکه فکر کنم تنهاماونو خاموش کردممیخواید یه نگاهی بهش بندازم؟نه، نه، به عامل انسانی نیاز داره...دکتر برند، کیس براتون از ایستگاهپیغامی ارسال کردهباشه، الان میام. ببخشیددکتر برند، متاسفانه پدرتون امروز فوت کردبا آرامش و بدون درد، درگذشتبهتون تسلیت میگممورف بود؟اون... اون بزرگ شدهبرند، میدونستی؟بهت گفته بود، نه؟میدونستیهمش یه دروغ بودتو ما رو ترک کردیکه خفه بشیمقحطی بکشیمپدرم هم میدونست؟!بابا،فقط میخوام بدونمتو منو اینجا گذاشتی که بمیرم؟فقط میخوام بدونم...کرده، من  A کوپر، پدرم کل زندیگشو صرف نقشهمن، هیچی از حرفاش سر در نمیارممن میدونماون اصلاً باور نداشت کهمردم رو از زمین نجات بده؟نهولی تعادل گرانش رو حل کنهآملیا، پدرت قبل از اینکه من برممشکل تعادل رو حل کرده بودولی چرا ازش استفاده نکرد؟تعادل نمی‌تونست نظریه نسبیت رو با مکانیککوانتومی تطبیق بده. بیشتر از این نیاز بودبیشتر؟ بیشتر چی؟،داده‌های بیشترمجبوری داخل سیاه چاله رو ببینیقوانین طبیعت، تکینگی گرانش رو منع کردهراملی، راست میگه؟،اگه سیاه چاله صدف باشهتکینگی مروارید اونهگرانش سیاه‌چاله به‌ قدری...نیرومنده که تکینگی همیشهفراتر از افق رویداد، مخفی می‌مونهبخاطر همینه که بهش "سیاه چاله" میگیمباشه، اگه بتونیم ماورایافق رویداد رو ببینیم چی؟نمی‌تونیم، کوپرچیزایی وجود دارن که باید مخفی باقی بموننپدرت مجبور بود راه‌ دیگه‌ای واسهنجات انسان‌ها از انقراض پیدا کنهمستعمره  ،B نقشهولی چرا به مردم نگفت؟چرا از ما مخفی کرد؟چون میدونست اگه این دروغ رو نگه...بجای اینکه مردم رومتحد برای این کار کنه، مردم فقط به جونخودشون فکر میکردن. یا به بچه‌هاشونمزخرفهخودت هیچ وقت بدون امید بهنجات انسان‌ها، اینجا نمیومدیتکامل نسل انسان‌ها باید فراتر از این درد باشهما میتونیم عمیقاً به عزیزانمون فکر کنیمولی احساس همدلی بندرت میتونهپیشرفت کار ما بشهولی دروغ؟دروغ وحشتناک؟نابخشودنیهو اون اینو می‌دونست. حاضر بود،انسانیت خودشو از دست بدهتا بتونه نسل انسان‌ها رو از انقراض نجات بدهفداکاری بزرگی کرده،نه، نه، فداکاری بزرگی کرد کهنسل انسان‌ها در زمین از بین بره،بخاطر غرور لعنتی‌اشامیدی در این قضیه ندیدمتاسفم، کوپرامیدی برای اونا نیستنه، نه -آینده، ماییم -کوپرکوپر، چیکار میتونم بکنم؟بذار برم خونهکاملاً مطمئنی؟راه‌حلش درستهسالهاست که به جواب رسیدهبدرد بخور نیست؟این نصفِ جوابهخب، نصف دیگه‌ش رو از کجا میخوای پیدا کنی؟،خارج از اینجا، توی سیاه‌چالهولی تو زمین حبس شدیمآره؟مطمئن نیستم بفهمیخدایا، اینا جمع‌ و جور کردن و میرنمثلاً میخوان کجا برن؟ -زنده بمونن -!تف به این روزگارمورف، بقیه حق ندارناین حقیقت رو بدونن؟ترس و هراس چاره‌ ساز نیستما باید مثل همیشه به کارمون ادامه بدیمآره، ولی خودِ پروفسور برند...از این کار منصرف شده بودبرند امیدشو از دست داده بودمن هنوزم روی این کار میکنمخب، ایده‌ای داری؟یه احساسقبلاً درمورد روح بهت گفته بودمپدرم فکر میکرد من بهش روح میگمچون ازش میترسمولی هیچ‌وقت ازش نمی‌ترسیدم،بهش روح میگفتم چون احساس میکردم،مثل یه انسانهکسی که سعی میکرد چیزی رو بهم بگه،اگه تو این دنیا جوابی برا این باشه،همونجاست،جایی در اون اتاقباید پیداش کنموقتمون رو تلف می‌کنیمتحویه‌های اکسیژن کمکی چی میشن؟اونا تاثیری به کیس ندارنمنم بیشتر راه رو تو خواب می‌گذرونمکوپ -بله؟ -یه پیشنهادی واسه برگشت از سفرت دارمچیه؟آخرین رخنه به سیاه چالهمن میرم خونه، رام -آره، میدونم -،تاثیر زمانی برات ندارهاین آخرین شانس واسه زمینی‌هاستبگو ببینمگارگنتوا" یه سیاه‌چاله‌ گردانِ قدیمی‌یه"واسه همین ما به تکینگی گرانشش "آرام" میگیمآرام؟اونا به ندرت آرام هستن، اما کششِ...گرانش به قدری سریعه کهاگه از افق رویداد سریع بگذری، ممکنهزنده بمونی. اینو یه کاوشگر گفته بوداگه از افق رد بشی چی میشه؟از افق رویداد به بعد چیزی معلوم نیستپس، فرض کنیم اگه احیاناً...راهی باشه که یه کاوشگر بتونهنگاهی اجمالی به تکینگی بندازه و داده‌هایکوانتومی رو به ما مخابره کنه؟اگه اون بتونه مجهز به انتقال...هر گونه‌ از انرژی که پالس میشه، باشه"از کِی تا حالا "اونمذکر شناخته میشه، پروفسور؟تارس" نامزد کاملاً مناسبیه"از قبل هم بهش گفتم دنبال چی بگردهمن به انتقال دهنده‌ی نوریکیپ" نیاز دارم، کوپر"اینو بخاطر ما میکنی؟قبل از اینکه احساساتی بشین...یادآوری میکنم من یه رباطمهرکاری بگین انجام میدمچراغت داره چشمک میزنهشوخی نمی‌کنممن به "تارس" نیاز دارم که چند قطعه مداراز "کیپ" برداره و به خودش نصب کنهخب، نمیخوام آسیبی بهاطلاعات بایگانی‌اش وارد بشهمن نظارت میکنمخیلی‌خبدکتر من، ما باید یکی برای آزمایشگاه برند، ،وقتی که اون ماژول‌ها فرود اومدننباید جایی برینخب، میتونم ببرمت به مناطقی که...گردشگرها رفتن ولی فکر نکنم،فکر نکنم این شرایط پایدار بمونهبهتره صبر کنیمکیس" چند دقیقه بعد با بقیه وسایل میاد اینجا"واقعاً میخوام اون مناطق روتا قبل از شب بررسی کنمخب، معمولاً طوفان‌های شدیدی ظاهر میشهباشه پسباید فرستنده‌ی دور بُرد داشته باشی -دارم -همه چی شارژه؟ -بله -دنبالم بیاتارس، فقط اطاعت، کوپر -برند بهم گفت که چرا میخوای برگردیولی کم لطفی کردم اگه اینو نگم...که همچین ماموریتیقطعاً به مهندسین بیشترینیاز خواهد داشتبهتره آروم بریاولویت در امنیته، کیس، یادت نرهاولویت در امنیت، کوپردکتر من، واقعاً باعث بسی افتخارهکه منم جزئی از این شدم...اما به محض اینکه کمپ‌ها رو ایجاد کردیم و،ماژول‌ها رو بررسی کردیم، کارم تمومهبرمیگردم خونههممون وابسته‌ایم،اما حتی بدون خانواده هم، بهت قول میدمآرزوی بودن با دیگران تحسین برانگیزهاین احساس، پایه و اساس چیزیه کهما رو به انسان تبدیل میکنهقابل تصرف نیستاز کِی سرفه میکنی؟ -مدتی هست -مامانم اجازه میده اینجا بازی کنمولی به وسایلت دست نزدمآروم میریم پایینخودت میدونی چرا ما نمیتونیم...روباط ها روبه اینجور ماموریت‌ها بفرستیم، مگه نه کوپر؟ماشین نمیتونه چاره‌‌‌ی موقتی پیدا کنه، چوننمیتونی ترس از مرگ رو در اونا جاسازی کنیغریزه‌ی زنده موندن‌مونتنها منبع بزرگ از الهامه،مثلاً خودت،یه پدر، با غریزه‌ زنده ماندنکه تا دیدن بچه‌هاش گسترش پیدا میکنهمیدونی آزمایشات درمورد قبلاز مُردن انسان چی میگه؟بچه‌هاتچهره‌هاشونموقع مرگ، ذهنت یکم بیشتر بهخودش فشار میاره که زنده بمونهبخاطر اونانفس عمیقشرط میبندم تو کوپیچند لحظه میتونی اینجا بشینیخیلی بدهاینا نمیتونن اینجا بموننخب؟ -باشه -تارس، چرا اینقدر طول کشید؟پروفسور، من نمیتونم بطور کامل راه اندازی کنمنمی‌فهمم؟جالبه،وقتی زمین رو ترک کردمفکر میکردم قراره دیر یا زود بمیرم،حقیقت اینه که من پیش شما نگفتمسیاره‌ی من واقعاً امکان حیات داره یا ندارههیچی طبق انتظارات من پیش نرفتبریمخیلی‌خب رفیق، حالا یه نفس عمیق بکشاین چیه؟ -چیکار میکنی؟ -شرمنده، نمیتونم بذارم با اون سفینه بری[B ما واسه تموم کردن ماموریت [نقشهبهش نیاز داریموقتی بقیه بفهمن اینجا امکان حیات نیستما نمیتونیم جون سالم بدر ببریمشرمنده. شرمندهاینا نمیتونن اینجا بمونن،شما باید از اینجا برین. رفیق، ببین...بذار یه چیزی رو رک بهت بگمتو مسئولیت داری -خدایا -کوپ، برو وسایلشو بیار، اون از اینجا میره!بابا تو رو تا این حد احمق به بار نیاورده، تامبابا منو بزرگ نکرده، پدربزرگ کرده...و اون هم مثلمامان و جسی، دفن شدهتمام اطلاعات این بود؟آرهاینجا حیاتی نیست؟نه. سعی کردم وظیفه‌مو انجام بدم، کوپرولی از وقتی که اومدم اینجا، میدونستماین مکان چیزی ندارهو سال‌ها این تنش رو تحمل کردم...ولی میدونستم، اگه اون دکمه رو بزنمیکی به کمکم خواهد آمد!ترسوی بی‌شرفبلهبلهبلهبله،ببین، اگه خودت نمیری، بذار خانوادت برهفقط بذار... خانوادت رو نجات بدهو برن زیرزمین زندگی کنن؟ با تو؟دعا کنن که بابایی دنیا رو نجات میده؟بابا برنمیگرده. هرگز برنمیگردهاین بر عهده منهمیخوای دنیا رو نجات بدی؟چونکه بابا نتونست اینکارو کنه؟!بــابــا حتی سعی نکرد!اون ما رو ترک کردما رو ترک کرد که اینجا بمیریم -هیچکی باهات نمیاد -میخوای مرگ بچه‌ت رو ببینی؟برو، و دیگه برنگردمیتونی اینا رو نگه ‌داری!بسهنه. نهدکتر من، هست که خودتو به کشتن بدیاینا بهترین احتمالاتی بودن که!سالها باهاش زنده موندمبهم دست نزن کوپرتو هرگز شرایط منو نگذروندیآدمای کمی این شرایط رو داشتنتو سعیتو کردی، مورفاحساس میکنی، نه؟غریزه زنده موندناینه که منو کشوند اینجا. اینه که همه ما روکشوند اینجا، و اینه که ما رو نجات میدهچون میخوام همه رو نجات بدمبخاطر تو، کوپر،متاسفم، نمیتونمنمیتونم اینجوری ببینمت. متاسفمفکر کردم میتونم، ولی نمیتونماینجام، بخاطر تو اینجامفقط به صدام گوش بدههمینجامتنها نیستیبچه‌هاتو می‌بینی؟چیزی نیستاونا پیش تو هستنقبل از اینکه بری، پروفسور برند اونشعر رو گفته بود؟یادت میاد؟"مرگِ آسان بر پیرمرد، مرگی شایسته نیست"چه خواهیم و چه نخواهیم""مرگ سراغ ما خواهد آمد"تا آخرین لحظاتِ سال‌خورده‌ات مقابله کن"!کمک! کمک -!کوپر -!کیس -!برند -!هوا نیست!کوپر. کوپر میام. کیس -تعمیر دارم -برو برو برو -رفـــتم -!کوپر، ما میاییم، تحمل کن، حرف نزنتا جایی که ممکنه کم نفس بکشخیلی نزدیک شدیمقربان، اینجا قفل امنیتی هست...به عامل انسانینیاز داره تا به عملکردهایش دسترسی پیدا کنمهمش ماله شما، قربانسعی کن نفس‌تو حبس کنیما میاییم، ما میاییمبجنب، زودباشسریع‌تر برو، سریع‌تر برو!کوپرتحمل کن، یالا!دیدمش! دیدمشبرو راست. برو راست!کوپرکوپر، اومدم!بگیر اینوجو، برو تانکرها رو بیاراین داده بی‌معنیهمتاسفم -چی؟ -من"، دروغ می‌گفت"!راملیراملی، صدامو میشنوی؟!راملیراملی -راملی صدامو میشنوی؟ -اینجا بمونلوئیسدرباره احتیاط چی بهت گفته بودم؟امنیت در اولویت، کوپرراملی، راملی صدامو می‌شنوی؟برند هستم!راملیدکتر برند، کوپرانفجاری رخ دادهمحفظه‌ی دکتر من!تارس، تارس، بپیچ چپهروقت تارس اومد، بهم بگوراملی نتونستنتونستم نجاتش بدمتارس اومد -از اینجا به بعد با من -موقعیت رنجر ثابته؟ -اون میخواد باهاش بره -اگه کنترل سفینه رو برعهده بگیره، بدبخت شدیمما رو ترک میکنه؟ -ما رو ترک میکنه -برو پایین. توی ماشین بمونیالا. سوار بشین!برو عقب بشین، حالادکتر من لطفاً جواب بدیندکتر من لطفاً جواب بدیناون طریقه‌ی اتصال به اندورنس رو نمیدونهولی خلبان ‌خودکار میدونه!نه تا وقتی که "تارس" خاموشش کنه!آفریندرجه اعتمادت چقدره، تارس؟ -!ظاهراً کمتر از تو -متصل نشوتکرار میکنم متصل نشواتصال اتوماتیک خاموش استبازنویسی کن -اجازه منع شد -!بازنویسی کناجازه منع شد،متصل نشید، تکرار میکنم...متصل نشید، لطفا جوابآروم میرم سمت اندورنساتصال ناتمامبازنویسی کنتونسته قفل کنه؟ -ناتمام -...دکتر منقفل دریچه آزاد شد،دکتر من، به‌هیچ وجه، تکرار میکنمدریچه رو اصلاً باز نکنیدتکرار میکنم، دریچه رو باز نکنید. اگه دریچه روباز کنید، فشار بر دریچه ‌هوا غلبه میکنهاگه دریچه ‌هوا رو منفجرکنه، چی؟اصلاً خوب نیستباشه، فاصله بگیر، فاصله بگیررانشگرها رو فعال کن، هرچی دم دست داریرانشگرها در حداکثر وضعیت -!برگرد عقب -...کیس، پیغام منو به کامپیوتر اصلی منتقل کنو بصورت پیغام اضطراری پخش کندکتر من، لطفا دریچه هوا رو باز نکنید...تکرار میکنم، دریچه رو بازبرند؟،نمیدونم اون بهت چی گفتهولی فرماندهی اندورنس رو بدست میگیرمو بعدش درمورد پایان دادنبه ماموریت حرف میزنیمدکتر من، گوش بدیناین درمورد زندگی من، یا زندگی کوپر نیستدرمورد تمام بشریته...لحظه‌ای هست...اینخدای من...کوپر، دلیلی نداره از سوختسرعت چرخش اندورنس رو محاسبه کنکوپر، چیکار میکنی؟ -بهش متصل میشم -سیستم میگه، موتورها رو با این چرخش تطبیق بدهغیرممکنهنه، ضروریهاندورنس داره وارد اتمسفر میشهاین محافظِ ضدگرما ندارهکیس، آماده‌ای؟ -آماده‌ام -کوپر، الان وقت احتیاط نیستکیس، اگه غش کنم، فرمون رو بدست بگیرتارس، آماده‌شو مکانیزم اتصال رو بیاری بیروناندورنس کم کم داره گرم میشه -باید سه درجه به راست بچرخی، کوپر -کوپر، هم راستا شدیمچرخش رو شروع کن!یالا تارس!یالا تارسمتصل شد -ببر بالا -آرومآرومرانشگرهاموتورهای اصلی فعال شداز مدار خارج میشیم. یالا عزیزمموتورها خاموشاز مدار خارج شدیم!بسیار عالی، حالا بریم به ترفند بعدیامیدوارم ترفند خوبی باشهبه سمت "گارگنتوا" کشیده می‌شیملعنت. کیس، فرمون رو بگیر -اطاعت -کوپر، ما به سمت "گارگنتوا" کشیده می‌شیماز موتورها استفاده کنم؟!نه، اجازه میدیم تا جایی که بلده ما رو بکشهگزارش بده -خبر خوب یا خبر بد؟ -خب، به جهنم، تارس!یه راست میریمژنراتور ذخیره فعال شده، سیستم پایدارههمه چی خوبهخوبه، باشه. قطب ناوبری به کل نابود شدهمنابع حیاتی بقدری نداریم که بتونیمبرگردیم سیاره زمینولی فکر کنم بتونیم به سیاره اِدموند برسیمسوخت چی؟کافی نیستولی یه نقشه دارم...اجازه میدیم "گارگنتوا" ما رو بکشه،به نزدیکی افق رویداد...با یه ضربه‌ی محکم از موتورمیتونه با فشار قوی پرتکنه به سمت سیاره اِدمونددستی؟واسه همین اینجاماین سفینه رو تا مدار بحران میبرماُفت زمان چی؟الان وقت نگرانی نسبیت زمان نیست، دکتر برندمتاسفم کوپر،وقتی دور "گارگنتوا" به سرعت کافی رسیدیم،ما از فرودبر ...و از رنجر برای فرار از گرانش سیاه‌چاله استفاده میکنیمکنترل از راه‌دور فرودبر نابود شدهواسه همین مجبوریم دستی این کارو کنیم،وقتی که سوخت فرودبر ...تارس جدا میشهو یه راست میره درون سیاه چالهواسه چی باید تارس جدا بشه؟باید وزن رو کم کنیم که از گرانش فرار کنیمقانون سوم نیوتنتنها راهی که انسان‌ها...میتونن چاره‌ای بیندیشناینه که باید چیزی رو پشت سر بگذارنکوپر، تو نمیتونی از تارس بخوایاینکارو واسه ما انجام بده...اون روباطهمجبور نیستی چیزی ازش بپرسی که انجام بدهکوپر، خیلی بی‌شعوریشرمنده که رنجوندمت -هدف نهایی ما اینه، دکتر برند -این تنها راه ما برای نجات آدمای زمینهاگه بتونم اونجا راهی واسه،انتقال داده‌های کوانتومی بکنمشاید اونا بتونن نجاتم بدنبذار هنوز هم امیدوار باشیم اونجاکسایی برای نجات وجود دارهبه حداکثر سرعت رسیدیمآماده برای فعال سازی رانشگرهاآماده‌ای؟ -آماده‌ام -اشتعال موتور اصلی در موتورهای فرودبر ‏موتورهای رنجر ‏!بهای این مانور کوچولو میشه !واسه فسیل ،فرودبر ‏جدا شدخدافظ تارسخدافظ دکتر برنداونطرف می‌بینمت، کوپاونجا می‌بینمت، قراضهخیلی‌خب، کیس. پرواز خوبی داشته باشیاز بهترین یاد گرفتیم،رنجر چی، نه، نه، کوپر، کوپرچیکار میکنی؟قانون سوم نیوتنباید چیزی پشت سر بذاریگفته بودی منابع کافی برای هردومون داریم...به توافق کرده بودیمنهجدا شوخیلی‌خب، آرام میرم پایینبه افق رویداد نزدیک می‌شمبه دهانه‌اش نزدیک میشم، به اعماقش میرم...وارد میشم... تاریکی -تصویری از افق دارم -همش تاریکهتارس، صدامو میشنوی؟تاریکی مطلقتارس صدامو میشنوی؟ تمامیه چیزایی می‌بینمتداخل ایجاد میشهکنترل فرمان رو از دست میدمفلش‌هایی مشاهده میکنمفلش‌هایی از روشنایی و تاریکیهرج و مرج گرانشی وجود دارهکامپیوترها از کار افتادنکشش گرانشیکنترل فرمان رو از دست دادم!مورف! مورف! بیا دیگه!مورف!مورف! مورف! مورف!مورفنه نه نه نه!مورف!نه! نه!اگه میخوای بری، پس برو -نه -!نه، نرو. نرو احمق! نروکد مورس، کد مورس"ب""م""ا"(خط تیره)(خط تیره)!مورف، ما وقت نداریم، یالا!بمان!یالا، یالا، مورف، مورف، یالاچی میگه؟ چی میگه؟ چی میگه؟"میگه "بمانبهش بگو، مورف. بگو بمونهوادارش کن، وادارش کن بمونهبمانوادارش کن بمونه، مورف!نذار برم، مورف!نذار، نذار برم مورفنه، نه، نهتو بودیتو روح من بودیکوپر، کوپر جواب بده -تارس؟ -دریافت شدنجات پیدا کردی؟ -...یه جایی -در فضای اونا ما رو نجات دادنآره؟ "اونا" کدوم خری هستن؟و چرا میخوان به ما کمک کنن؟... نمیدونم، ولی اونا این فضای درون فضای باعث میشه تو درک کنیآره، ولی کارساز نیستچرا هست. دیدی که زمان بصورتیک بُعدِ فیزیکی ارائه شدتلاش کردی تا نیرویی رو از فضا-زمان وارد کنی...گرانشکه باهاش پیغام بفرستم؟ -بله -...گرانشمیتونه از ابعاد عبور کنه، حتی زمان؟(زمان بُعد چهارم است)ظاهراًداده کوانتومی رو داری؟ -دریافت شد، بله دارم -...طول‌ تمامی امواجش رو ترجمه کردمولی چیزی به بیرون درج نمیکنه -من میتونم، میتونم -ولی چنین داده‌‌ پیچیده‌ای برای بچه؟ -الان دیگه بچه نیست -خودتو نشون بدهمورف، آتیش داره نزدیک میشه، بجنبحتی اگه از اینجا باهاش ارتباط برقرار کنیاون نمیتونه سال‌ها به معنیش پی ببرهنه، نه، فهمیدم تارس، ولی بایدیه فکری بهش بکنیموگرنه آدمای زمین می‌میرنفکر کن، فکر کن، فکر کنکوپر، اونا ما رو اینجانیاوردن که گذشته رو تغییر بدیمبازم بگواونا ما رو اینجا نیاوردنکه گذشته رو تغییر بدیمنه، اونا ما رو اصلاً اینجا نیاوردنما خودمونو آوردیمتارس، مختصات ناسا رو بصورت باینری بهم بدهباینری، دریافت شد، ترجمه داده‌هاروح نیستگرانشه -هنوز نگرفتی، تارس؟ -!من خودمو آوردم اینجاما اینجاییم که با دنیای سهبُعدی ارتباط برقرار کنیمما پل ارتباطیمفکر میکردم "اونا" منو انتخاب کردن،ولی اونا منو انتخاب نکردناونا دختره رو انتخاب کردنبرای چی، کوپر؟برای نجات دنیاهمه‌ی اینا، فقط اتاق دختر رو نشون میدهتمام لحظاتش، بی‌نهایت پیچیده استاونا" به فضا و زمان نامحدود دسترسی دارن"ولی موقعیت ارتباط رو نمیدونناونا نمیتونن موقعیت خاصی از زمان رو پیدا کننواسه همین نمی‌تونن ارتباط برقرار کنن،واسه همینه که اینجاماینجام که به طریقی به مورف بگمولی باید اون موقعیت خاص رو پیدا کنمچطوری کوپر؟عشق، تارسعشق،همونطور که برند گفته بودارتباطم با مورف، قابل سنجشهکلید ارتباط همینه -اینجا قراره چیکار کنیم؟ -راهی پیدا کنم که بهش بگمساعتساعتخودشهداده‌ها رو باید به بُعد پایین‌تر کد گذاری کنمتارس، داده‌ها رو به کد مورس ترجمه کنو بدش به منترجمه میکنم به کد مورسکوپر، اگه دنبال ساعت نیاد چی؟میاد. میاد!مورف، ماشینو دیدم!داره میادباشه، میام پاییناز کجا میدونی؟چون ساعتو خودم دادم بهش:مورس دریافت شدنقطه، نقطه، خط تیره، نقطهنقطه، نقطه، خط تیره، نقطهنقطه، خط تیره، نقطه، نقطهنقطه، نقطه، خط تیره، نقطهخودشه!تمام این وقت!می‌دونستم، اونهبابا نجاتمون داد!یافتمرسمش اینه!یافتمکار کرد؟ -فکر کنم آره -از کجا میدونی؟چون گونه‌های بیگانهدارن مکعب رو می‌بندنبازم نگرفتی، تارس؟اونا گونه‌های بیگانه نیستناونا"، ما انسان‌ها هستیم"،کاری که با مورف میکنماونا" هم با من میکنن"برای همه ماکوپر، انسان‌ها نمی‌توننچنین چیزی رو درست کنننه، نه. هنوز نهولی یه روزی میتوننمن و تو نمی‌تونیم. ولی انسان‌هاتمدنی فوق‌پیشرفته از انسان‌ها که دردنیایی فراتر از حالا چی میشه؟آقای کوپرعجله نکنید، قربانخیلی آروم، آقای کوپریادتون باشه، دیگه جوان بی‌تجربه نیستینشما در واقع آروم قربانشما، بشدت خوش‌شانس بودینکاوشگرها شما رو تنها با اکسیژن باقی‌مونده در لباستون پیدا کردنکجام؟ایستگاه فضایی کوپرالان دور زحل می‌چرخیمایستگاه کوپر؟چه اسم جالبی بعد از من گذاشتینچیه؟راستش، اسمتون رو بعد از،ترک شما نذاشتیمبعد از دخترتون این اسم رو گذاشتیمدرضمن همیشه یادآوری میکرد کهشما چقدر مهم بودینهنوزم زنده هست؟دو هفته بعد، میرسه اینجاواسه منتقل کردنش از یه،ایستگاه دیگه، بیش از حد پیرهاما وقتی خبر پیدا شدنتون رو شنیدخب، دیگه مهم‌تر از این چی میتونه باشهموضوع "مورفی و کوپر" ـهبله اینطورهدو روز بعد مرخص میشیدمطمئنم از دیدن وسایل خونه حیرت زده میشید،راستش، درمورد شما، موقع دبیرستانمقاله‌ای نوشته بودمراجع به زندگیتون در زمین خیلی چیزا میدونستمبله، درسته،قربان، اگه بشه دنبالم بیاییدموقعیت خیلی خوبی پیش رو دارین،وقتی پیشنهادم رو به خانم کوپر دادمبا کمال میل پیشنهادم رو قبول کردنمنم خیلی خوشحال شدمگرچه باهاش رو در رو صحبت نکردمپدرم یک کشاورز بودمثل همه که مجبور به این کار بودنبهرحال، غذای کافی نبودبینی و دهانمون رو با پارچه‌هایی می‌پوشوندیمکه نتونیم گرد و خاک زیادی رو بدمیمایشون تایید کردن که شماخیلی کشاورزی رو دوست دارینبلاخره تایید کرد؟!خب، هیچی مثل خونه نمیشههمه چیز مثل اصلش چیده و طراحی شدهبلهاین؟بله، ماشینی بود که نزدیک زحلکنار شما پیدا کردیممنبع قدرتش تموم شدهولی اگه خواستین میتونیم دوباره روشنش کنیم!بله، خواهش میکنمتنظیماتتنظیمات کلیتنظیمات امنیتصداقت، تنظیم جدید%تایید شد. تغییرات اضافی؟%شوخ طبع، تایید شد... تغییر تنظیم بصورت خودکار، %اونو بکن ‏‏تاخ تاخ -دلت واقعاً شبیه این بود؟من به این تمیزی نبودم، قراضهچندان اهمیت نمیدم که خودمونو وانمودکنیم به جایی برگشتیم که از اونجا شروع کردیممیخوام بدونم الان کجاییم؟کجا میریم؟آقای کوپر، تمام خانواده اینجاستخانواده؟بله، همشون به دیدنش اومدنایشون به مدت بهشون گفتی من کشاورزی رو دوست دارم؟منم مورفمن روح تو بودممیدونممردم نمی‌دونست، فکر میکردن...همشو خودم کردم. امامیدونستم کار کی بودکسی بهم باور نکردولی میدونستم برمیگردیچطور؟چونکه بابام بهم قول دادهاینجام مورفاینجامنه. هیچ خانواده‌ای نباید شاهد مرگ فرزندش بشهنوه‌های خودمو دارمبروکجا؟برنداون، یه جایی در بین ستاره‌هاستکمپ درست میکنهتنها، در کهکشانی بیگانه...شاید الان خودشو برایخواب طولانی آماده میکنهبنا به روشنایی خورشید جدیدماندر خانه‌ی جدیدمان'
keyword_count = 10

In [ ]:
normalizer = Normalizer()
normalize_text = normalizer.normalize(text)
tokenize_text = [word_tokenize(txt) for txt in sent_tokenize(normalize_text)]
tokenize_text